In [ ]:
%%bash

apt-get update && \
apt-get upgrade -y && \
apt-get install -y --no-install-recommends aria2

download_model() {
    local url="$1"
    local full_path="$2"

    local destination_dir destination_file
    destination_dir=$(dirname "$full_path")
    destination_file=$(basename "$full_path")

    mkdir -p "$destination_dir"

    # Check if file already exists
    if [[ -f "$full_path" ]]; then
        echo "INFO: ✅ $destination_file already exists, skipping download."
        return 0
    fi

    # Clean up orphaned aria2 control files
    for control_file in "${full_path}.aria2" "${full_path}.__temp"; do
        if [[ -f "$control_file" ]]; then
            echo "INFO: 🗑️ Removing stale control file: $control_file"
            rm -f "$control_file"
        fi
    done

    echo "INFO: 📥 Downloading $destination_file to $destination_dir..."

    # aria2c options
    local aria2_opts=(
        -x 16                           # 16 connections per server (max bandwidth)
        -s 16                           # 16 segments per file (better for large files)
        -k 5M                           # 5MB chunk size
        --continue=true                 # Resume partial downloads
        --max-tries=5                   # Retry up to 5 times
        --retry-wait=2                  # Wait 2 seconds between retries
        --min-split-size=25M            # Split files only if >25MB
        --max-overall-download-limit=0  # No overall speed limit
        --max-download-limit=0          # No per-connection speed limit
        --console-log-level=warn        # Show warnings/errors only
        --download-result=full          # Show full download summary
        --auto-file-renaming=false      # Don’t rename files on conflict
        --allow-overwrite=true          # Overwrite existing files (set false to avoid)
        --check-certificate=true        # Verify SSL certificates
        --enable-http-keep-alive=true   # Keep HTTP connections alive
        --enable-http-pipelining=true   # Enable HTTP pipelining
        --header="Accept: */*"          # Accept all content types
        -d "$destination_dir"           # Download directory
        -o "$destination_file"          # Output filename
    )

    # Background download
    aria2c "${aria2_opts[@]}" "$url" &
    local download_pid=$!

    echo "INFO: 🚀  Download started in background for $destination_file (PID: $download_pid)"
}

if [ -z "$NETWORK_VOLUME" ]; then
  NETWORK_VOLUME="/"
  MODELS_PATH="${COMFYUI_HOME%/}/models"
else
  MODELS_PATH="${NETWORK_VOLUME%/}/models"
fi


CLIP="$MODELS_PATH/clip"
PULID="$MODELS_PATH/pulid"
VEA="$MODELS_PATH/vae/"
UNET="$MODELS_PATH/unet/"
LORAS="$MODELS_PATH/loras/"
CONTROL_NET="$MODELS_PATH/controlnet/"
CHECKPOINTS="$MODELS_PATH/checkpoints/"
CLIP_VISION="$MODELS_PATH/clip_vision/"
SAMS="$MODELS_PATH/sams"
UPSCALERS="$MODELS_PATH/upscale_models"
SEGM="$COMFYUI_HOME/models/ultralytics/segm"
BBOXES="$COMFYUI_HOME/models/ultralytics/bbox"


mkdir -p "$CLIP" "$PULID" "$VEA" "$UNET" "$LORAS" "$CONTROL_NET" "$CHECKPOINTS" \
"$CLIP_VISION" $SAMS" "$UPSCALERS" "$SEGM" "$BBOXES"


# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# VAEs
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Flux AE VAE
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/vae/flux/ae.safetensors" "$VEA/ae.safetensors"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# CLIPs
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# CLIP-L
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/clip/clip_l.safetensors" "$CLIP/clip_l.safetensors"

# T5-XXL - fp8
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/clip/t5xxl_fp8_e4m3fn_scaled.safetensors" "$CLIP/t5xxl_fp8_e4m3fn_scaled.safetensors"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Clip vision
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# CLIP-VIT-L
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/clip-vision/clip-vit-large-patch14.safetensors" "$CLIP_VISION/clip-vit-large-patch14.safetensors"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# LoRAs
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Photorealistic Skin LoRA - Trigger - aidmarealisticskin
download_model "https://civitai.com/api/download/models/1301668?type=Model&format=SafeTensor" "$LORAS/flux-realistic-skin-lora-v0.1.safetensors"

# Hourglass Body -  Trigger - extremely tiny waist, thick legs, extremely wide hips, extremely slim hourglass body, h0u4 
download_model "https://civitai.com/api/download/models/1376192?type=Model&format=SafeTensor" "$LORAS/hourglass-body-flux-lora.safetensors"

# Perfect Full Round Breasts & Slim Waist - Trigger woman
download_model "https://civitai.com/api/download/models/1782533?type=Model&format=SafeTensor" "$LORAS/perfect-breasts-slim-waist-flux-lora.safetensors"

# amateurphoto-v6.safetensors - No Trigger
download_model "https://civitai.com/api/download/models/993999?type=Model&format=SafeTensor" "$LORAS/flux-amateurphoto-v6-lora.safetensors"

# SameFace_Fix.safetensors - Trigger - woman
download_model "https://civitai.com/api/download/models/993999?type=Model&format=SafeTensor" "$LORAS/flux-same-face-fix-lora.safetensors"


# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# ControlNets
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# ShakkerLabs Union Pro 2.0
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/control-net/flux/flux-dev-shakker-labs-controlnet-union-pro-2.0" "$CONTROL_NET/flux-dev-shakker-labs-controlnet-union-pro-2.0"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# PuLID
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Flux PulID v0.9.1
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/pulid/pulid_flux_v0.9.1.safetensors" "$PULID/pulid_flux_v0.9.1.safetensors"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Upscalers
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# 4x-ClearRealityV1
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/upscalers/4x-ClearRealityV1.pth" "$UPSCALERS/4x-ClearRealityV1.pth"

# 8xNMKDSuperscale_150000G
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/upscalers/8xNMKDSuperscale_150000G.pt" "$UPSCALERS/8xNMKDSuperscale_150000G.pt"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# SAM
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# SAM VIT-L
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/sam/sam_vit_l_0b3195.pth" "$SAMS/sam_vit_l_0b3195.pth"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# BBOXs
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Face
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/bbox/face_yolov9c.pt" "$BBOXES/face_yolov9c.pt"

# Eye
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/bbox/eyeful_individual_yolov8.pt" "$BBOXES/eyeful_individual_yolov8.pt"

# Lips
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/bbox/lips-v1.pt" "$BBOXES/lips-v1.pt"

# Hand
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/bbox/hand_yolov9c.pt" "$BBOXES/hand_yolov9c.pt"

# NudeNet: lightweight Nudity detection
download_model "https://huggingface.co/SimonJoz/nudenet/resolve/main/640m.pt" "$BBOXES/nudenet-640m.pt"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# SEGM
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# face_bisenetv1_celebhq_79999
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/segm/face_bisenetv1_celebhq_79999.pth" "$SEGM/face_bisenetv1_celebhq_79999.pth"

# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Checkpoints
# ──────────────────────────────────────────────────────────────────────────────────────────────────────
# Flux.1 Dev
# download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/checkpoint/flux/flux1-dev.safetensors" "$UNET/flux1-dev.safetensors"

# Flux.1 Dev - Perfect Base
# download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/checkpoint/flux/flux1-dev-perfect-base.safetensors" "$UNET/flux1-dev-perfect-base.safetensors"

# Flux.1 Dev - Real Dream
download_model "https://civitai.com/api/download/models/1703341?type=Model&format=SafeTensor&size=full&fp=fp8" "$UNET/flux1-dev-real-dream.safetensors"

# Flux.1 Dev - STOIQO NewReality
# download_model "https://civitai.com/api/download/models/979329?type=Model&format=SafeTensor&size=full&fp=fp16" "$UNET/flux1-STOIQO-NewReality_F1DAlpha.safetensors"

# Flux.1 Dev - STOIQOAfroditeFLUXXL
# download_model "https://civitai.com/api/download/models/897489?type=Model&format=SafeTensor&size=full&fp=fp16" "$UNET/flux1-STOIQO-AfroditeFLUXXL_F1DAlpha.safetensors"

# Flux.1 Dev Kontext
download_model "https://huggingface.co/SimonJoz/comfy/resolve/main/checkpoint/flux/flux1-kontext-dev.safetensors" "$UNET/flux1-kontext-dev.safetensors"

